In [34]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from pattern.es import parse, split
from nltk.corpus import stopwords
from collections import defaultdict
import unidecode

In [35]:
data = pd.read_csv("Data\\preguntas_jp.csv", sep=',', encoding = "ISO-8859-1")

In [36]:
data.head(5)

,Pregunta,Tema,Pers/empresa,Label
0,"Buenos dìas, necesito servicios de provisiòn d...",Jubilacion Patronal,empresa,Cotizacion
1,"Buenos Dias, favor necesito un estudio actuari...",Jubilacion Patronal,empresa,Cotizacion
2,Buenos días... Estimada Daniela le saludos de ...,Jubilacion Patronal,empresa,Ambigous
3,Buenos dìas necesito servicios de provisiòn de...,Jubilacion Patronal,empresa,Cotizacion
4,buenos dias cual seria el costo del estudio pa...,Jubilacion Patronal,empresa,Cotizacion


In [37]:
p2 = []
for p in data.Pregunta:
    p2.append(' '.join(p.split()))
data['Pregunta2'] = p2

In [38]:
s = parse(data.Pregunta2[30].lower(), lemmata = True).split()[0]
s

[['solicité', 'NN', 'B-NP', 'O', 'solicité'],
 ['una', 'DT', 'I-NP', 'O', 'un'],
 ['cotización', 'NN', 'I-NP', 'O', 'cotización'],
 ['y', 'CC', 'O', 'O', 'y'],
 ['me', 'PRP', 'B-NP', 'O', 'me'],
 ['enviaron', 'VB', 'B-VP', 'O', 'enviar'],
 ['dos', 'CD', 'B-NP', 'O', 'dos'],
 ['precios', 'NNS', 'I-NP', 'O', 'precio'],
 ['una', 'DT', 'O', 'O', 'un'],
 ['como', 'IN', 'B-PP', 'B-PNP', 'como'],
 ['niif', 'NN', 'B-NP', 'I-PNP', 'niif'],
 ['y', 'CC', 'O', 'O', 'y'],
 ['otra', 'DT', 'O', 'O', 'otra'],
 ['cómo', 'WP$', 'O', 'O', 'cómo'],
 ['nec', 'NN', 'B-NP', 'O', 'nec'],
 [',', ',', 'O', 'O', ','],
 ['cuál', 'WP$', 'O', 'O', 'cuál'],
 ['es', 'VB', 'B-VP', 'O', 'ser'],
 ['la', 'DT', 'B-NP', 'O', 'el'],
 ['diferencia', 'NN', 'I-NP', 'O', 'diferencia'],
 ['por', 'IN', 'B-PP', 'B-PNP', 'por'],
 ['favor', 'IN', 'I-PP', 'I-PNP', 'favor'],
 ['me', 'PRP', 'B-NP', 'I-PNP', 'me'],
 ['podrían', 'VB', 'B-VP', 'O', 'poder'],
 ['aclarar', 'VB', 'I-VP', 'O', 'aclarar'],
 ['.', '.', 'O', 'O', '.']]

In [39]:
def position(l, b, element):
    try:
        p = l[b:].index(element)
        return p
    except:
        return -1

In [40]:
def cmin(*args):
    a = np.array(list(args))
    a = a[a>=0]
    if len(a)>0:
        return np.min(a)
    else:
        return -1

In [41]:
l = [x[1] for x in s]
position(l,0,'VB')

5

In [42]:
p1 = -1
p2 = -1
p3 = -1
cmin(p1,p2,p3)

-1

In [43]:
def ext_algo1(sentence):  
    words = []
    
    for s in parse(sentence.lower(), lemmata = True).split():        
   
        tags = [x[1] for x in s]
        b = 0
        
        while b < len(tags) - 1:  
            w_g = []
            p = position(tags,b,'VB')
            if p == -1:
                break
            else:        
                w_g.append(s[b+p][4])
                b = b + p + 1
                while True:
                    p1 = position(tags,b,'VB')
                    p2 = position(tags,b,'NN')
                    p3 = position(tags,b,'NNS')
                    pm = cmin(p1,p2,p3)
            
                    if pm == -1:
                        b = len(tags)
                        break            
                    elif p1 == pm:
                        w_g.append(s[b+p1][4])#global pos
                        b = b + p1 + 1
                    elif p2 == pm:
                        w_g.append(s[b+p2][4])
                        b = b + p2 + 1
                        break
                    else:
                        w_g.append(s[b+p3][4])
                        b = b + p3 + 1
                        break                 
                words.append(w_g)
        
    return words   

In [44]:
def ext_algo2(sentence):  
    words = []
    
    for s in parse(sentence.lower(), lemmata = True).split():        
   
        tags = [x[1] for x in s]
        b = 0
        
        while b < len(tags) - 1:  
            w_g = []
            p_1 = position(tags,b,'WP$')
            p_2 = position(tags,b,'IN')
            p = cmin(p_1,p_2)            
            if p == -1:
                break
            else:        
                w_g.append(s[b+p][4])
                b = b + p + 1
                while True:
                    p1 = position(tags,b,'VB')
                    p2 = position(tags,b,'NN')
                    p3 = position(tags,b,'NNS')
                    pm = cmin(p1,p2,p3)
            
                    if pm == -1:
                        b = len(tags)
                        break            
                    elif p1 == pm:
                        w_g.append(s[b+p1][4])#global pos
                        b = b + p1 + 1
                    elif p2 == pm:
                        w_g.append(s[b+p2][4])
                        b = b + p2 + 1
                        break
                    else:
                        w_g.append(s[b+p3][4])
                        b = b + p3 + 1
                        break                 
                words.append(w_g)
        
    return words   

In [45]:
data.Pregunta2[28]

'ggarofalo@cargoline.ec gardenia garofalo de la empresa Cargoline del Ecuador S.A. ..Por favor indicarme el costo por la estimación actuarial JUBILACION Y DESAHUCIO de 4 empleados.'

In [46]:
s = parse(data.Pregunta2[24].lower(), lemmata = True).split()[0]
s

[['necesito', 'VB', 'B-VP', 'O', 'necesitar'],
 ['me', 'PRP', 'B-NP', 'O', 'me'],
 ['ayude', 'VB', 'B-VP', 'O', 'ayudar'],
 ['con', 'IN', 'B-PP', 'O', 'con'],
 ['informacion', 'VB', 'B-VP', 'O', 'informacion'],
 ['tengo', 'VB', 'I-VP', 'O', 'tener'],
 ['una', 'DT', 'O', 'O', 'un'],
 ['nomina', 'VB', 'B-VP', 'O', 'nominar'],
 ['de', 'IN', 'B-PP', 'B-PNP', 'de'],
 ['18', 'CD', 'B-NP', 'I-PNP', '18'],
 ['trabajadores', 'NNS', 'I-NP', 'I-PNP', 'trabajador'],
 ['pero', 'CC', 'O', 'O', 'pero'],
 ['dentro', 'IN', 'B-PP', 'B-PNP', 'dentro'],
 ['de', 'IN', 'I-PP', 'I-PNP', 'de'],
 ['ellos', 'PRP', 'B-NP', 'I-PNP', 'ellos'],
 ['un', 'DT', 'I-NP', 'I-PNP', 'un'],
 ['persona', 'NN', 'I-NP', 'I-PNP', 'persona'],
 ['de', 'IN', 'B-PP', 'B-PNP', 'de'],
 ['80años', 'NN', 'B-NP', 'I-PNP', '80años'],
 ['que', 'WP$', 'O', 'O', 'que'],
 ['necesito', 'VB', 'B-VP', 'O', 'necesitar'],
 ['jubilarlo', 'NN', 'B-NP', 'O', 'jubilarlo'],
 ['cual', 'IN', 'B-PP', 'O', 'cual'],
 ['seria', 'JJ', 'B-ADJP', 'O', 'serio']

In [47]:
ext_algo1(data.Pregunta2[23])

[['necesitar', 'ayudar', 'informacion', 'tener', 'nominar', 'trabajador'],
 ['necesitar', 'jubilarlo'],
 ['calcular']]

In [48]:
ext_algo2(data.Pregunta2[24])

[['con', 'informacion', 'tener', 'nominar', 'trabajador'],
 ['dentro', 'persona'],
 ['de', '80años'],
 ['que', 'necesitar', 'jubilarlo'],
 ['cual', 'costo'],
 ['si', 'ayudaraia'],
 ['con', 'calcular']]

In [49]:
def class_palabra(lista):
    tokens = ['costo',  'cotización', 'cotizar',  'cotizacion',  'precio']
    r = 0
    for l in lista:
        if len(l)>0:
            for w in l:
                if w in tokens:            
                    r = 1
                    
    tokens2 = ['necesitar','requerir','actualizar']
    tokens3 = ['estudio', 'servicio']
    
    for l in lista:
        if len(l) == 2:
            if l[0] in tokens2 and l[1] in tokens3:
                r =1                    
        
    return r
        

In [50]:
def algo_clasificador(sentence):
    lista1 = ext_algo1(sentence)
    lista2 = ext_algo2(sentence)
    lista = lista1 + lista2
    r = class_palabra(lista)
    if r == 1:
        return 'Cotizacion'
    else:
        return 'Otra Cosa'

In [51]:
data['result'] =[algo_clasificador(x) for x in data.Pregunta2]

In [52]:
data

,Pregunta,Tema,Pers/empresa,Label,Pregunta2,result
0,"Buenos dìas, necesito servicios de provisiòn d...",Jubilacion Patronal,empresa,Cotizacion,"Buenos dìas, necesito servicios de provisiòn d...",Cotizacion
1,"Buenos Dias, favor necesito un estudio actuari...",Jubilacion Patronal,empresa,Cotizacion,"Buenos Dias, favor necesito un estudio actuari...",Cotizacion
2,Buenos días... Estimada Daniela le saludos de ...,Jubilacion Patronal,empresa,Ambigous,Buenos días... Estimada Daniela le saludos de ...,Otra Cosa
3,Buenos dìas necesito servicios de provisiòn de...,Jubilacion Patronal,empresa,Cotizacion,Buenos dìas necesito servicios de provisiòn de...,Cotizacion
4,buenos dias cual seria el costo del estudio pa...,Jubilacion Patronal,empresa,Cotizacion,buenos dias cual seria el costo del estudio pa...,Cotizacion
5,Le saludo de la firma de auditoria Mackliff Au...,Jubilacion Patronal,empresa,Cotizacion,Le saludo de la firma de auditoria Mackliff Au...,Otra Cosa
6,"BUENAS TARDES, NECESITO QUE ME AYUDEN CON UN E...",Jubilacion Patronal,empresa,Cotizacion,"BUENAS TARDES, NECESITO QUE ME AYUDEN CON UN E...",Cotizacion
7,"Estimados, Quiero saber mas sobre el servicio ...",Jubilacion Patronal,empresa,Cotizacion,"Estimados, Quiero saber mas sobre el servicio ...",Otra Cosa
8,Quisiera una cotización de un estudio actuarial,Jubilacion Patronal,empresa,Cotizacion,Quisiera una cotización de un estudio actuarial,Cotizacion
9,"Le saluda Claudia Camino de ZHM Seguros, nosot...",Jubilacion Patronal,empresa,Cotizacion,"Le saluda Claudia Camino de ZHM Seguros, nosot...",Cotizacion


In [53]:
data[data['result']=='Cotizacion']

,Pregunta,Tema,Pers/empresa,Label,Pregunta2,result
0,"Buenos dìas, necesito servicios de provisiòn d...",Jubilacion Patronal,empresa,Cotizacion,"Buenos dìas, necesito servicios de provisiòn d...",Cotizacion
1,"Buenos Dias, favor necesito un estudio actuari...",Jubilacion Patronal,empresa,Cotizacion,"Buenos Dias, favor necesito un estudio actuari...",Cotizacion
3,Buenos dìas necesito servicios de provisiòn de...,Jubilacion Patronal,empresa,Cotizacion,Buenos dìas necesito servicios de provisiòn de...,Cotizacion
4,buenos dias cual seria el costo del estudio pa...,Jubilacion Patronal,empresa,Cotizacion,buenos dias cual seria el costo del estudio pa...,Cotizacion
6,"BUENAS TARDES, NECESITO QUE ME AYUDEN CON UN E...",Jubilacion Patronal,empresa,Cotizacion,"BUENAS TARDES, NECESITO QUE ME AYUDEN CON UN E...",Cotizacion
8,Quisiera una cotización de un estudio actuarial,Jubilacion Patronal,empresa,Cotizacion,Quisiera una cotización de un estudio actuarial,Cotizacion
9,"Le saluda Claudia Camino de ZHM Seguros, nosot...",Jubilacion Patronal,empresa,Cotizacion,"Le saluda Claudia Camino de ZHM Seguros, nosot...",Cotizacion
10,"Buen dia, disculpe quisiera saber cual es el c...",Jubilacion Patronal,empresa,Cotizacion,"Buen dia, disculpe quisiera saber cual es el c...",Cotizacion
12,"Por favor, a qué telefono le llamo para solici...",Jubilacion Patronal,empresa,Cotizacion,"Por favor, a qué telefono le llamo para solici...",Cotizacion
14,necesito cotización para un estudio actuarial....,Jubilacion Patronal,empresa,Cotizacion,necesito cotización para un estudio actuarial....,Cotizacion


In [54]:
ext_algo1(data.Pregunta2[3])

[['necesitar', 'servicio']]

In [55]:
data.Pregunta2[3]

'Buenos dìas necesito servicios de provisiòn de jubilaciòn del personal'

In [56]:
data[(data['Label']=='Cotizacion') & (data['result']!='Cotizacion')]

,Pregunta,Tema,Pers/empresa,Label,Pregunta2,result
5,Le saludo de la firma de auditoria Mackliff Au...,Jubilacion Patronal,empresa,Cotizacion,Le saludo de la firma de auditoria Mackliff Au...,Otra Cosa
7,"Estimados, Quiero saber mas sobre el servicio ...",Jubilacion Patronal,empresa,Cotizacion,"Estimados, Quiero saber mas sobre el servicio ...",Otra Cosa
21,Buenas tardes Me gustaría saber si ustedes pod...,Jubilacion Patronal,empresa,Cotizacion,Buenas tardes Me gustaría saber si ustedes pod...,Otra Cosa
26,Buenas tardes cuanto me costaría realizar un c...,Jubilacion Patronal,empresa,Cotizacion,Buenas tardes cuanto me costaría realizar un c...,Otra Cosa
28,ggarofalo@cargoline.ec gardenia garofalo de la...,Jubilacion Patronal,empresa,Cotizacion,ggarofalo@cargoline.ec gardenia garofalo de la...,Otra Cosa
39,Buenas tardes hace unos dias solicite la cotiz...,Jubilacion Patronal,empresa,Cotizacion,Buenas tardes hace unos dias solicite la cotiz...,Otra Cosa


In [ ]:
Queria estudio, Solicite cotizacion

In [ ]:
5,28,39,41